In [1]:
import io
import os
import sqlite3
import zipfile
from pathlib import Path

import pandas as pd
import regex
from tqdm import tqdm

In [2]:
DATA_DIR = Path('..') / 'data'
RAW_DIR = DATA_DIR / '00_raw'
INTERIM_DIR = DATA_DIR / '01_interim'

ZIP = RAW_DIR / 'iDigBio_snapshot_2021-01.zip'
ZIP_CSV = 'occurrence_raw.csv'

OUT_CSV = INTERIM_DIR / 'idigbio_2021-01_taxonomy_2021-03-16a.csv'

CHUNK = 1_000_000

In [3]:
def get_csv_headers():
    """Get the headers from the CSV file in the zipped snapshot."""
    with zipfile.ZipFile(ZIP) as zip_file:
        with zip_file.open(ZIP_CSV) as csv_file:
            headers = csv_file.readline()
    return [h.decode().strip() for h in sorted(headers.split(b','))]

In [4]:
# get_csv_headers()

In [5]:
COLUMNS = {
    'coreid': 'coreid',
    'dwc:taxonRank': 'taxonRank',
    'dwc:taxonomicStatus': 'taxonomicStatus',
    'dwc:kingdom': 'kingdom',
    'dwc:phylum': 'phylum',
    'dwc:class': 'class_',
    'dwc:order': 'order_',
    'dwc:genus': 'genus',
    'dwc:subgenus': 'subgenus',
    'dwc:specificEpithet': 'specificEpithet',
    'dwc:infraspecificEpithet': 'infraspecificEpithet',
    'dwc:scientificName': 'scientificName',
    'symbiota:verbatimScientificName': 'verbatimScientificName',
    'dwc:scientificNameAuthorship': 'scientificNameAuthorship',
}

In [8]:
def new_csv():
    with zipfile.ZipFile(ZIP) as zippy:
        with zippy.open(ZIP_CSV) as csv_file:

            reader = pd.read_csv(
                csv_file, dtype=str, keep_default_na=False,
                chunksize=CHUNK, usecols=COLUMNS.keys())

            first_chunk = True

            for df in tqdm(reader):
                df = df.rename(columns=COLUMNS)

                if first_chunk:
                    df.to_csv(OUT_CSV, index=False)
                else:
                    df.to_csv(OUT_CSV, index=False, mode='a', header=False)

                first_chunk = False

In [9]:
new_csv()

127it [1:05:03, 30.73s/it]


split --additional-suffix=.csv --numeric-suffixes=1 --lines=50000000 idigbio_2021-01_taxonomy_2021-03-16a.csv idigbio_2021-01_taxonomy_2021-03-16a